<a href="https://colab.research.google.com/github/ILPATHERA/ML2_Pokemoncreator/blob/main/ML2_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My Machine Learning 2 Image Classification Model

### Import Libraries

In [ ]:
import pandas as pd

### Load Data

In [ ]:
data = https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types

### Transforming the Data

### Creating Custom Dataset and DataLoading

In [ ]:
train_data_path = "/kaggle/input/image-classification-screening/train"
test_data_path = "/kaggle/input/image-classification-screening/test"


train_img_paths = []

for data_path in glob.glob(train_data_path+'/*'):
    #print(data_path)
    train_img_paths.append(glob.glob(data_path))
    
train_img_paths = [x[0] for x in train_img_paths]
random.shuffle(train_img_paths)

train_img_paths , valid_img_paths = train_img_paths[:int(0.8* len(train_img_paths))], train_img_paths[int(0.8*len(train_img_paths)):]


test_img_paths = []
for data_path in glob.glob(test_data_path + '/*'):
    test_img_paths.append(glob.glob(data_path))

test_img_paths = [x[0] for x in test_img_paths]
print("Train size: {}\nValid size: {}\nTest size: {}".format(len(train_img_paths), len(valid_img_paths), len(test_img_paths)))

### Visualizing Results





## Model

### Defining Layers

### Training